In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Transformer 만들기

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [4]:
# 최종 버전
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스(2i)에는 사인 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])

    # 배열의 홀수 인덱스(2i+1)에는 코사인 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:, 0::2] = sines
    angle_rads[:, 1::2] = cosines
    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    print(pos_encoding.shape)
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [5]:
def scaled_dot_product_attention(query, key, value, mask):
  # query 크기 : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  # key 크기 : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
  # value 크기 : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
  # padding_mask : (batch_size, 1, 1, key의 문장 길이)

  # Q와 K의 곱. 어텐션 스코어 행렬.
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 스케일링
  # dk의 루트값으로 나눠준다.
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 마스킹. 어텐션 스코어 행렬의 마스킹 할 위치에 매우 작은 음수값을 넣는다.
  # 매우 작은 값이므로 소프트맥스 함수를 지나면 행렬의 해당 위치의 값은 0이 된다.
  if mask is not None:
    logits += (mask * -1e9)

  # 소프트맥스 함수는 마지막 차원인 key의 문장 길이 방향으로 수행된다.
  # attention weight : (batch_size, num_heads, query의 문장 길이, key의 문장 길이)
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # output : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  output = tf.matmul(attention_weights, value)

  return output, attention_weights

In [6]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    # d_model을 num_heads로 나눈 값.
    # 논문 기준 : 64
    self.depth = d_model // self.num_heads

    # WQ, WK, WV에 해당하는 밀집층 정의
    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    # WO에 해당하는 밀집층 정의
    self.dense = tf.keras.layers.Dense(units=d_model)

  # num_heads 개수만큼 q, k, v를 split하는 함수
  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # 1. WQ, WK, WV에 해당하는 밀집층 지나기
    # q : (batch_size, query의 문장 길이, d_model)
    # k : (batch_size, key의 문장 길이, d_model)
    # v : (batch_size, value의 문장 길이, d_model)
    # 참고) 인코더(k, v)-디코더(q) 어텐션에서는 query 길이와 key, value의 길이는 다를 수 있다.
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 2. 헤드 나누기
    # q : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    # k : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
    # v : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
    # (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
    # (batch_size, query의 문장 길이, num_heads, d_model/num_heads)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 4. 헤드 연결(concatenate)하기
    # (batch_size, query의 문장 길이, d_model)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 5. WO에 해당하는 밀집층 지나기
    # (batch_size, query의 문장 길이, d_model)
    outputs = self.dense(concat_attention)

    return outputs

In [7]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, key의 문장 길이)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [8]:
def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 멀티-헤드 어텐션 (첫번째 서브층 / 셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': padding_mask # 패딩 마스크 사용
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 포지션 와이즈 피드 포워드 신경망 (두번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [9]:
def encoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 인코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [10]:
# 디코더의 첫번째 서브층(sublayer)에서 미래 토큰을 Mask하는 함수
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x) # 패딩 마스크도 포함
  return tf.maximum(look_ahead_mask, padding_mask)

In [11]:
def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

  # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 멀티-헤드 어텐션 (첫번째 서브층 / 마스크드 셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': look_ahead_mask # 룩어헤드 마스크
      })

  # 잔차 연결과 층 정규화
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 멀티-헤드 어텐션 (두번째 서브층 / 디코더-인코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1, 'key': enc_outputs, 'value': enc_outputs, # Q != K = V
          'mask': padding_mask # 패딩 마스크
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 포지션 와이즈 피드 포워드 신경망 (세번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [12]:
def decoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')

  # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 디코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [13]:
def transformer(vocab_size, num_layers, dff,
                d_model, num_heads, dropout,
                name="transformer"):

  def from_config(cls, config):
      return cls(**config)
  # 인코더의 입력
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 디코더의 입력
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더의 패딩 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더의 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 디코더의 패딩 마스크(두번째 서브층)
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

      
  # 인코더의 출력은 enc_outputs. 디코더로 전달된다.
  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[inputs, enc_padding_mask]) # 인코더의 입력은 입력 문장과 패딩 마스크

  # 디코더의 출력은 dec_outputs. 출력층으로 전달된다.
  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 다음 단어 예측을 위한 출력층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [14]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [15]:

# small_transformer = transformer(
#     vocab_size = 9000,
#     num_layers = 4,
#     dff = 512,
#     d_model = 128,
#     num_heads = 4,
#     dropout = 0.3,
#     name="small_transformer")

# tf.keras.utils.plot_model(
#     small_transformer, to_file='small_transformer.png', show_shapes=True)

In [16]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

# 데이터 전처리

train data에서 질문 데이터의 수와 답변 데이터의 수가 달라서, 이를 1대 1로 맞춰주기 위해 전처리 진행

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Dataset/chatbot/chatbot_train.csv')
test = pd.read_csv('/content/drive/MyDrive/Dataset/chatbot/chatbot_test.csv')

In [ ]:
train.head()

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트,가격,수량,크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
0,1,c,아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?,의류,67435,q,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,밤 12시 30분,NaN,NaN,67435,1
1,2,s,네 저희가 보통 그날 12시에 발주 마감이여서요 일단 내일 물건 오는 대로 보내드릴게요.,의류,67435,a,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,12시,NaN,NaN,67435,2
2,3,c,실수로 취소하면 재주문해야 하는 거죠?,의류,34557,q,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34557,1
3,4,s,네 취소하였을 경우 재주문해 주셔야 하는 점 양해 부탁드립니다.,의류,34557,a,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34557,2
4,5,c,택배비 따로 추가되나요?,의류,17,q,m,배송_비용_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2186,33


In [ ]:
train_q = train[train['발화자'] == 'c']
train_a = train[train['발화자'] == 's']
test_q = test[test['발화자'] == 'c']
test_a = test[test['발화자'] == 's']

In [ ]:
train_q.head()

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트,가격,수량,크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
0,1,c,아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?,의류,67435,q,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,밤 12시 30분,NaN,NaN,67435,1
2,3,c,실수로 취소하면 재주문해야 하는 거죠?,의류,34557,q,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34557,1
4,5,c,택배비 따로 추가되나요?,의류,17,q,m,배송_비용_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2186,33
6,7,c,택배비 있나요?,의류,20,q,m,배송_비용_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,296,39
8,9,c,택배비 따로 들어요,의류,40,q,m,배송_비용_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2057,88


In [ ]:
train_a.head()

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트,가격,수량,크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
1,2,s,네 저희가 보통 그날 12시에 발주 마감이여서요 일단 내일 물건 오는 대로 보내드릴게요.,의류,67435,a,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,12시,NaN,NaN,67435,2
3,4,s,네 취소하였을 경우 재주문해 주셔야 하는 점 양해 부탁드립니다.,의류,34557,a,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34557,2
5,6,s,아뇨 택배비는 따로 추가없습니다,의류,17,a,m,배송_비용_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2186,34
7,8,s,택배비는 저희 무상입니다,의류,20,a,m,배송_비용_질문,무상,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,296,40
9,10,s,아니요 택배비는 저희 무상으로 하고 있어서 택배비는 따로 안듭니다,의류,40,a,m,배송_비용_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2057,89


In [ ]:
train_a['count'] = 0
train_q['count'] = 0

test_a['count'] = 0
test_q['count'] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
idx = list(train_a['QA번호'].value_counts()[train_a['QA번호'].value_counts() == 1].index)
idx_test = list(test_a['QA번호'].value_counts()[test_a['QA번호'].value_counts() == 1].index)

In [ ]:
for i in range(len(train_a)):
  if train_a.iloc[i,4] in idx:
    train_a.iloc[i,-1] += 1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
for i in range(len(test_a)):
  if test_a.iloc[i,4] in idx_test:
    test_a.iloc[i,-1] += 1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
for i in range(len(test_q)):
  if test_q.iloc[i,4] in idx_test:
    test_q.iloc[i,-1] += 1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
for i in range(len(train_q)):
  if train_q.iloc[i,4] in idx:
    train_q.iloc[i,-1] += 1

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
train_a.to_csv('/content/drive/MyDrive/Dataset/chatbot/train_answer.csv', index = False, encoding='cp949')
train_q.to_csv('/content/drive/MyDrive/Dataset/chatbot/train_question.csv', index = False, encoding='cp949')

In [ ]:
train_q = train_q[train_q['count'] == 1]
train_a = train_a[train_a['count'] == 1]

In [ ]:
train_a = pd.read_csv('/content/train_answer.csv', encoding = 'cp949')

In [ ]:
train_a

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트,가격,수량,크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번,count
0,2,s,네 저희가 보통 그날 12시에 발주 마감이여서요 일단 내일 물건 오는 대로 보내드릴게요.,의류,67435,a,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,12시,NaN,NaN,67435,2,1
1,4,s,네 취소하였을 경우 재주문해 주셔야 하는 점 양해 부탁드립니다.,의류,34557,a,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34557,2,1
2,46,s,입금 확인되어 익일 상품 꼼꼼히 검수하여 최대한 빠른 출고도와 드리겠습니다.,의류,32351,a,m,배송_비용_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32351,2,1
3,48,s,"네, 40000원부터 무료 배송이에요.",의류,121724,a,m,배송_비용_질문,40000원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121724,2,1
4,50,s,두 벌 이상 구매하시면 무료로 해드리고 있습니다.,의류,106149,a,m,배송_비용_질문,NaN,두 벌,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106149,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160237,419420,s,32로 확인되십니다.,의류,37194,a,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37194,2,1
160238,419422,s,현재 가격 수정되신 것으로 확인됩니다. 다시 확인하시어 구매하여 주시기 바랍니다.,의류,40429,a,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40429,2,1
160239,419424,s,받아보신 상품이 프리 사이즈가 맞습니다 동봉된 택은 제작공장에서 표기된 걸로 저희 ...,의류,64694,a,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64694,2,1
160240,419426,s,두 가지 상품 전부입고 지연 상품이라 발송해 드리지 못하고 있는 것 같아요 한 가지...,의류,46330,a,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46330,2,1


In [ ]:
train_q

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트,가격,수량,크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번,count
0,1,c,아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?,의류,67435,q,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,밤 12시 30분,NaN,NaN,67435,1,1
2,3,c,실수로 취소하면 재주문해야 하는 거죠?,의류,34557,q,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34557,1,1
44,45,c,5시에 입금 일괄 확인하는 거면 통화할 때 그렇게 말해줘야 하는 거 아닌가요?,의류,32351,q,m,배송_비용_질문,NaN,NaN,NaN,NaN,NaN,NaN,5시,NaN,NaN,32351,1,1
46,47,c,40000원부터 무료 배송이죠?,의류,121724,q,m,배송_비용_질문,40000원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121724,1,1
48,49,c,택배비는 얼마인가요?,의류,106149,q,m,배송_비용_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106149,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419418,419419,c,15번 트레이닝 세트윗옷 가슴 단면 32라고 적혀 있는데 잘못 적힌 거 아닌가요?,의류,37194,q,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,트레이닝,37194,1,1
419420,419421,c,뭐가 수정이 됐단 거죠?,의류,40429,q,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40429,1,1
419422,419423,c,11번 잠옷 프리 사이즈 주문해서 물건받았는데요 사이즈택에 XXx L라고 되어 있네...,의류,64694,q,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,잠옷,64694,1,1
419424,419425,c,어제는 해주겠지 했는데 또 안 했네요?,의류,46330,q,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46330,1,1


In [ ]:
train_a.to_csv('/content/drive/MyDrive/Dataset/chatbot/train_answer.csv', index = False, encoding='cp949')
train_q.to_csv('/content/drive/MyDrive/Dataset/chatbot/train_question.csv', index = False, encoding='cp949')

In [ ]:
test_q['count'].value_counts()

1    26940
Name: count, dtype: int64

# 챗봇 드가자!

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf

In [18]:
train_a = pd.read_csv('/content/drive/MyDrive/Dataset/chatbot/train_answer.csv', encoding = 'cp949')
train_q = pd.read_csv('/content/drive/MyDrive/Dataset/chatbot/train_question.csv', encoding = 'cp949')

test_a = pd.read_csv('/content/drive/MyDrive/Dataset/chatbot/test_answer.csv', encoding = 'cp949')
test_q = pd.read_csv('/content/drive/MyDrive/Dataset/chatbot/test_question.csv', encoding = 'cp949')

In [ ]:
train_q

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트,가격,수량,크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번,count
0,1,c,아침도 아니고 밤 12시 30분에 결제한 건데도 그런가요?,의류,67435,q,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,밤 12시 30분,NaN,NaN,67435,1,1
1,3,c,실수로 취소하면 재주문해야 하는 거죠?,의류,34557,q,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34557,1,1
2,45,c,5시에 입금 일괄 확인하는 거면 통화할 때 그렇게 말해줘야 하는 거 아닌가요?,의류,32351,q,m,배송_비용_질문,NaN,NaN,NaN,NaN,NaN,NaN,5시,NaN,NaN,32351,1,1
3,47,c,40000원부터 무료 배송이죠?,의류,121724,q,m,배송_비용_질문,40000원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121724,1,1
4,49,c,택배비는 얼마인가요?,의류,106149,q,m,배송_비용_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106149,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160237,419419,c,15번 트레이닝 세트윗옷 가슴 단면 32라고 적혀 있는데 잘못 적힌 거 아닌가요?,의류,37194,q,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,트레이닝,37194,1,1
160238,419421,c,뭐가 수정이 됐단 거죠?,의류,40429,q,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40429,1,1
160239,419423,c,11번 잠옷 프리 사이즈 주문해서 물건받았는데요 사이즈택에 XXx L라고 되어 있네...,의류,64694,q,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,잠옷,64694,1,1
160240,419425,c,어제는 해주겠지 했는데 또 안 했네요?,의류,46330,q,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46330,1,1


In [ ]:
train_a

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트,가격,수량,크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번,count
0,2,s,네 저희가 보통 그날 12시에 발주 마감이여서요 일단 내일 물건 오는 대로 보내드릴게요.,의류,67435,a,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,12시,NaN,NaN,67435,2,1
1,4,s,네 취소하였을 경우 재주문해 주셔야 하는 점 양해 부탁드립니다.,의류,34557,a,m,주문_취소_확인,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34557,2,1
2,46,s,입금 확인되어 익일 상품 꼼꼼히 검수하여 최대한 빠른 출고도와 드리겠습니다.,의류,32351,a,m,배송_비용_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32351,2,1
3,48,s,"네, 40000원부터 무료 배송이에요.",의류,121724,a,m,배송_비용_질문,40000원,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,121724,2,1
4,50,s,두 벌 이상 구매하시면 무료로 해드리고 있습니다.,의류,106149,a,m,배송_비용_질문,NaN,두 벌,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106149,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160237,419420,s,32로 확인되십니다.,의류,37194,a,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37194,2,1
160238,419422,s,현재 가격 수정되신 것으로 확인됩니다. 다시 확인하시어 구매하여 주시기 바랍니다.,의류,40429,a,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40429,2,1
160239,419424,s,받아보신 상품이 프리 사이즈가 맞습니다 동봉된 택은 제작공장에서 표기된 걸로 저희 ...,의류,64694,a,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64694,2,1
160240,419426,s,두 가지 상품 전부입고 지연 상품이라 발송해 드리지 못하고 있는 것 같아요 한 가지...,의류,46330,a,m,주문_오류_질문,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46330,2,1


In [19]:
df = pd.concat([train_q['발화문'], train_a['발화문'], train_a['인텐트']], axis = 1)
df.columns = ['question', 'answer', 'label']

In [20]:
questions = []
for sentence in df['question']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

In [21]:
answers = []
for sentence in df['answer']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    # sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [22]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)

In [23]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2

In [24]:
MAX_LENGTH = 25

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2)

  # 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs

In [25]:
questions, answers = tokenize_and_filter(questions, answers)

In [26]:
# 텐서플로우 dataset을 이용하여 셔플(shuffle)을 수행하되, 배치 크기로 데이터를 묶는다.
# 또한 이 과정에서 교사 강요(teacher forcing)을 사용하기 위해서 디코더의 입력과 실제값 시퀀스를 구성한다.
BATCH_SIZE = 256
BUFFER_SIZE = 20000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {
        'outputs': answers[:, 1:]  # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다.
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [28]:
D_MODEL = 512
NUM_LAYERS = 6
NUM_HEADS = 16
DFF = 512
DROPOUT = 0.3

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

(1, 8139, 512)
(1, 8139, 512)


In [29]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [ ]:
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
626/626 [==============================] - 189s 279ms/step - loss: 4.0048 - accuracy: 0.0844
Epoch 2/50
626/626 [==============================] - 174s 278ms/step - loss: 3.0115 - accuracy: 0.1414
Epoch 3/50
626/626 [==============================] - 174s 278ms/step - loss: 2.4772 - accuracy: 0.1754
Epoch 4/50
626/626 [==============================] - 174s 278ms/step - loss: 2.1887 - accuracy: 0.1981
Epoch 5/50
626/626 [==============================] - 174s 278ms/step - loss: 2.0185 - accuracy: 0.2129
Epoch 6/50
626/626 [==============================] - 174s 278ms/step - loss: 1.9107 - accuracy: 0.2232
Epoch 7/50
626/626 [==============================] - 174s 278ms/step - loss: 1.8324 - accuracy: 0.2307
Epoch 8/50
626/626 [==============================] - 174s 278ms/step - loss: 1.7470 - accuracy: 0.2395
Epoch 9/50
626/626 [==============================] - 174s 278ms/step - loss: 1.6719 - accuracy: 0.2479
Epoch 10/50
626/626 [==============================] - 174s 278m

KeyboardInterrupt: ignored

In [ ]:

# I suggest You try the following:

# model = tf.keras.Model(...)
model.save_weights("/content/drive/MyDrive/Dataset/chatbot/model_save/model_D_MODEL=512_Layer=4_Heads=16_Dropout=0.3_fin.h5")
# ...
# model.load_weights("some_path")

In [ ]:
D_MODEL = 512
NUM_LAYERS = 4
NUM_HEADS = 16
DFF = 512
DROPOUT = 0.3

# 챗봇 기능 검증

In [ ]:
model.load_weights("/content/model.h5")

In [ ]:
def preprocess_sentence(sentence):
  # 단어와 구두점 사이에 공백 추가.
  # ex) 12시 땡! -> 12시 땡 !
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

In [ ]:
def evaluate(sentence):
  # 입력 문장에 대한 전처리
  sentence = preprocess_sentence(sentence)

  # 입력 문장에 시작 토큰과 종료 토큰을 추가
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # 현재 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 현재 시점의 예측 단어를 output(출력)에 연결한다.
    # output은 for문의 다음 루프에서 디코더의 입력이 된다.
    output = tf.concat([output, predicted_id], axis=-1)

  # 단어 예측이 모두 끝났다면 output을 리턴.
  return tf.squeeze(output, axis=0)

In [ ]:
def predict(sentence):
  prediction = evaluate(sentence)

  # prediction == 디코더가 리턴한 챗봇의 대답에 해당하는 정수 시퀀스
  # tokenizer.decode()를 통해 정수 시퀀스를 문자열로 디코딩.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
output = predict("지금 당장 환불해 주세요")

Input: 지금 당장 환불해 주세요
Output: 반품 상품 입고 확인 후 빠른 환불 처리도와 드리도록 할게요 .


In [ ]:
output = predict("지금 당장 환불해 주세요")

Input: 지금 당장 환불해 주세요
Output: 네 해당 상품은 현재 입고 지연으로 인해 배송이 지연되고 있습니다 .  입고 되는 대로 최대한 빠르게 출고해 드리겠습니다 .


In [ ]:
output = predict("배송이 너무 늦어요")

Input: 배송이 너무 늦어요
Output: 주문하신 상품 중 일부 입고 지연되고 있어 이번 주까지 시간 양해 부탁드리며 기다려주시면 재입고되는 대로 최대한 빠르게 출고도와 드리겠습니다 .


In [ ]:
output = predict("배송이 너무 늦어요")

Input: 배송이 너무 늦어요
Output: 주문하신 상품 금일 출고 예정이며 빠르면 내일 수령 가능하시나 택배 물량 지역에 따라 배송이 지연될수 있는 점 양해 부탁드려요 .


In [ ]:
output = predict("왜 배송이 늦나요?")

Input: 왜 배송이 늦나요?
Output: 주문하신 상품 금일 출고 예정입니다 택배사의 사정에 따라 2 - 3일 소요될 수 있는 점 양해 부탁드립니다 .


In [ ]:
output = predict("이 제품을 교환하고 싶어요")

Input: 이 제품을 교환하고 싶어요
Output: 네 교환 상품 입고 후 교환 진행하도록 하겠습니다 .


In [ ]:
output = predict("이 제품을 교환하고 싶어요")

Input: 이 제품을 교환하고 싶어요
Output: 네 ,  해당 제품은 완판되어 있는 제품으로 교환해드립니다 .


In [ ]:
output = predict("제 피부가 건성인데 괜찮을까요?")

Input: 제 피부가 건성인데 괜찮을까요?
Output: 네 ,  이 레깅스는 스판력이 좋은 제품입니다 .
